# Run SourceXtractor++ in detection mode

This notebook runs SE++ in detection mode, meaning it will use a detection image (can be one of the images used for measurements, or a composite image such as the weigthed sum of all your images) to detect the sources before performing model fitting. 

For this notebook, you must have [SourceXtractor++](https://sourcextractorplusplus.readthedocs.io/en/latest/) installed on your machine.

In [2]:
import os
import glob
import re

Here, we select the parameters, and especially the Python configuration file, the images to use (for measurement and detection), and where to save the output catalog and the model and residual images (checkimages).

In [3]:
# Python configuration file for SE++
python_config='/home/ec2-user/DAWN/DJA-SEpp/config/sepp-config.py'

# Field to run SE++ on (for file organization)
field = 'GDS'

# Detection image
detect_img=glob.glob(f'/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*ir*sci_cutout.fits')[0]

# Filter to use on the detection image
filt='/home/ec2-user/DAWN/DJA-SEpp/config/gauss_1.5_3x3.conv'

# Band filters to use for SE++ (here, automatically detected from the available images)
# filter_list = list(set([re.search('(f\d+\w+)', filename).group(1) for filename in glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*clear*sci*.fits")]))
filter_list = ['f090w', 'f150w', 'f277w', 'f444w']
filter_list.sort()
print(f"Filters : {filter_list}")

# Arguments to give to SE++ :
#       - fit_case : the model for fitting in SE++ (keep 'sersic_rg4' for detection mode)
#       - list_of_IMG_names : list of images to use (here, automatically detected using 'filter_list')
#       - list_of_WHT_names : list of weight images to use (here, automatically detected using 'filter_list')
#       - list_of_PSF_names : list of PSF files to use (here, automatically detected using 'filter_list')
args = {'fit_case' : 'sersic_rg4',
        'list_of_IMG_names' : [glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*{filter}*sci_cutout.fits")[0] for filter in filter_list],
        'list_of_WHT_names' : [glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/image/{field}/*{filter}*wht_cutout.fits")[0] for filter in filter_list],
        'list_of_PSF_names' : [glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/psfex/{field}/*{filter}*star_psf.psf")[0] for filter in filter_list],
        }
print(f"Images  : {args['list_of_IMG_names']}")
print(f"Weights : {args['list_of_WHT_names']}")
print(f"PSFs    : {args['list_of_PSF_names']}")

# Output
output_dir = "/home/ec2-user/DAWN/DJA-SEpp/sepp/benchmark"
properties='PixelCentroid,WorldCentroid,SourceIDs,GroupInfo,GroupStamp,SourceFlags,NDetectedPixels,NCorePixel,AperturePhotometry,AutoPhotometry,FluxRadius,SNRRatio,ShapeParameters,FlexibleModelFitting'
name = ".".join(detect_img.split("/")[-1].split(".")[:-1])
output_catname=f'{output_dir}/{field}/{name}_sepp_cat.fits'
checkimg_path=f'{output_dir}/{field}/checkimages'
os.makedirs(checkimg_path, exist_ok=True)

Filters : ['f090w', 'f150w', 'f277w', 'f444w']
Images  : ['/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f090w-clear_drc_sci_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f150w-clear_drc_sci_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f277w-clear_drc_sci_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f444w-clear_drc_sci_cutout.fits']
Weights : ['/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f090w-clear_drc_wht_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f150w-clear_drc_wht_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f277w-clear_drc_wht_cutout.fits', '/home/ec2-user/DAWN/DJA-SEpp/image/GDS/gds-grizli-v7.2-f444w-clear_drc_wht_cutout.fits']
PSFs    : ['/home/ec2-user/DAWN/DJA-SEpp/psfex/GDS/GDS-f090w-clear_drc_cat_star_psf.psf', '/home/ec2-user/DAWN/DJA-SEpp/psfex/GDS/GDS-f150w-clear_drc_cat_star_psf.psf', '/home/ec2-user/DAWN/DJA-SEpp/psfex/GDS/GD

The following line runs SE++, and can be configured with many different parameters. The parameters given here should work for most cases, but feel free to contact me or look at the the SE++ documentation (or through the `sourcextractor++ --help` command) for more help.

In [4]:
# os.environ['TMPDIR'] = '/FlashStorage/tmp'
os.system(f"sourcextractor++ \
         --python-config-file {python_config} \
         --python-arg '{args}' \
         --output-catalog-filename {output_catname} \
         --output-properties {properties} \
         --output-flush-size 10 \
         --detection-image {detect_img} \
         --weight-type WEIGHT \
         --weight-absolute 0 \
         --segmentation-filter {filt} \
         --background-cell-size 128 \
         --segmentation-algorithm LUTZ \
         --smoothing-box-size 5 \
         --detection-threshold 0.80 \
         --detection-minimum-area 7 \
         --partition-corethreshold yes \
         --core-threshold-value 1.5 \
         --core-minimum-area 9 \
         --grouping-algorithm split \
         --partition-multithreshold yes \
         --partition-minimum-area 18 \
         --grouping-hard-limit 0 \
         --partition-minimum-contrast 0.0001 \
         --partition-threshold-count 42 \
         --use-cleaning yes \
         --cleaning-minimum-area 15 \
         --model-fitting-iterations 550 \
         --sampling-scale-factor 1 \
         --check-image-model-fitting {checkimg_path}/model.fits \
         --check-image-residual {checkimg_path}/resid.fits \
         --log-file {output_catname.replace('.fits', '.log')} \
         --log-level INFO \
         --thread-count 128 \
         --tile-memory-limit 4096 \
         --tile-size 2048 \
         ")


2024-05-13T11:13:39UTC Config  INFO : Loaded segmentation filter: /home/ec2-user/DAWN/DJA-SEpp/config/gauss_1.5_3x3.conv width: 3 height: 3
2024-05-13T11:13:39UTC Python::stderr  INFO : /home/ec2-user/miniconda3/envs/dawn-310/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
2024-05-13T11:13:39UTC Python::stderr  INFO :   setattr(self, word, getattr(machar, word).flat[0])
2024-05-13T11:13:39UTC Python::stderr  INFO : /home/ec2-user/miniconda3/envs/dawn-310/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
2024-05-13T11:13:39UTC Python::stderr  INFO :   return self._float_to_str(self.smallest_subnormal)
2024-05-13T11:13:39UTC Python::stdout  INFO : F090W
2024-05-13T11:13:39UTC Python::stdout  INFO : F150W
2024-05-13T11:13:39UTC Python::stdout  INFO : F277W
2024-05-13T11:13:39UTC Python::stdout

2